# 1.5 「ファインチューニング」で精度向上を実現する方法

- 本ファイルでは、学習済みのVGGモデルを使用し、ファインチューニングでアリとハチの画像を分類するモデルを学習します



# 学習目標

1.	PyTorchでGPUを使用する実装コードを書けるようになる
2.	最適化手法の設定において、層ごとに異なる学習率を設定したファインチューニングを実装できるようになる
3.	学習したネットワークを保存・ロードできるようになる



# 事前準備

- 1.4節で解説したAWS EC2 のGPUインスタンスを使用します


In [1]:
# パッケージのimport
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms


In [2]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)


# DatasetとDataLoaderを作成

In [25]:

from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

size=224

mean=(0.485,0.456,0.406)
std=(0.229,0.224,0.225)

train_dataset = HymenopteraDataset(file_list=train_list, transform=ImageTransform(size, mean,std), phase="train")
val_dataset = HymenopteraDataset(file_list=val_list, transform=ImageTransform(size, mean,std), phase="val")

batch_size=32

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}



./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


# ネットワークモデルの作成

In [26]:
use_pretrained = True

net = models.vgg16(pretrained=use_pretrained)

net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

net.train()

print("ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました")

ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました


# 損失関数を定義

In [27]:
# 損失関数の設定
criterion=nn.CrossEntropyLoss()

# 最適化手法を設定

In [28]:
params_to_update_1 =[]
params_to_update_2 =[]
params_to_update_3 =[]


update_param_names_1 =["features"]
update_param_names_2 =["classifier.0.weight", "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 =["classifier.6.weight", "classifier.6.bias"]

for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("add params_to_update1", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("add params_to_update2", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("add params_to_update3", name)

    else:
        param.requires_grad = False
        print("勾配計算なし、学習しない:", name)
        


add params_to_update1 features.0.weight
add params_to_update1 features.0.bias
add params_to_update1 features.2.weight
add params_to_update1 features.2.bias
add params_to_update1 features.5.weight
add params_to_update1 features.5.bias
add params_to_update1 features.7.weight
add params_to_update1 features.7.bias
add params_to_update1 features.10.weight
add params_to_update1 features.10.bias
add params_to_update1 features.12.weight
add params_to_update1 features.12.bias
add params_to_update1 features.14.weight
add params_to_update1 features.14.bias
add params_to_update1 features.17.weight
add params_to_update1 features.17.bias
add params_to_update1 features.19.weight
add params_to_update1 features.19.bias
add params_to_update1 features.21.weight
add params_to_update1 features.21.bias
add params_to_update1 features.24.weight
add params_to_update1 features.24.bias
add params_to_update1 features.26.weight
add params_to_update1 features.26.bias
add params_to_update1 features.28.weight
add par

In [29]:
# 最適化手法の設定
optimizer=optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)


# 学習・検証を実施

In [34]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス", device)
    net.to(device)
    
    torch.backends.cudnn.benchmark = True
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1,num_epochs))
        print('-------------')
        
        for phase in ["train","val"]:
            if phase == "train":
                net.train()
            elif phase =="val":
                net.eval()
            
            epoch_loss = 0.0
            epoch_corrects = 0
            
            if(epoch == 0) and (phase =='train'):
                continue
                
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase=='train'):
                    outputs=net(inputs)
                    loss=criterion(outputs, labels)
                    _,preds = torch.max(outputs,1)
                    
                    if phase=='train':
                        loss.backward()
                        optimizer.step()
                        
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
                    
            
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double()/ len(dataloaders_dict[phase].dataset)
            print('{} Loss:{:.4f} Acc:{:.4f}'.format(phase, epoch_loss, epoch_acc))
                

In [35]:
# 学習・検証を実行する
num_epochs=2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)


  0%|          | 0/5 [00:00<?, ?it/s]

使用デバイス cuda:0
Epoch 1/2
-------------


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss:0.6652 Acc:0.5163
Epoch 2/2
-------------


  0%|          | 0/5 [00:00<?, ?it/s]

train Loss:0.4536 Acc:0.7613


100%|██████████| 5/5 [00:01<00:00,  2.67it/s]

val Loss:0.1738 Acc:0.9542


# 学習したネットワークを保存・ロード

In [37]:
# PyTorchのネットワークパラメータの保存
save_path = 'weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)



In [40]:
# PyTorchのネットワークパラメータのロード
load_path = 'weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# GPU上で保存された重みをCPU上でロードする場合
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)


<All keys matched successfully>

以上